## Imports

In [1]:
import math 
import random
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
from transformers import RobertaTokenizer, RobertaForMaskedLM
from typing import List
from torch.utils.data import DataLoader, Dataset
from model import RobertaClassificationAndLM
from model2 import RobertaClassificationAndLM2
from data import EthicsDataset, MoralStoriesDataset
from datasets import load_dataset
from tqdm import tqdm


if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
    
print(f"using device: {device}")

torch.set_float32_matmul_precision('high')

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

using device: mps


In [45]:
import math 
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as n
from transformers import RobertaForMaskedLM



#####################################
#      Building Block Classes       #
#####################################
class RobertaEmbeddings(nn.Module):
   
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)

        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
      
        self.padding_idx = config.pad_token_id
        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size, padding_idx=self.padding_idx
        )

    def forward(self, input_ids):

        def create_position_ids_from_input_ids(input_ids, padding_idx):
        
            mask = input_ids.ne(padding_idx).int()
            incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
            return incremental_indices.long() + padding_idx
    
        position_ids = create_position_ids_from_input_ids(input_ids, self.padding_idx)
            
        embeddings = self.word_embeddings(input_ids)
                
        position_embeddings = self.position_embeddings(position_ids)
        
        embeddings += position_embeddings
        
        embeddings = self.LayerNorm(embeddings)
        
        return embeddings
    

class RobertaSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
       
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)


    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward( self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)
    
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))
        query_layer = self.transpose_for_scores(mixed_query_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in RobertaModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(new_context_layer_shape)

        outputs =  context_layer

        return outputs

class RobertaSdpaSelfAttention(RobertaSelfAttention):
    def __init__(self, config):
        super().__init__(config)

    def forward( self, hidden_states, attention_mask = None):
        
        bsz, tgt_len, _ = hidden_states.size()

        query_layer = self.transpose_for_scores(self.query(hidden_states))
        
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        attn_output = torch.nn.functional.scaled_dot_product_attention(
            query_layer,
            key_layer,
            value_layer,
            attn_mask=attention_mask,
            dropout_p=0.0,
            is_causal=False,
        )

        attn_output = attn_output.transpose(1, 2)
        attn_output = attn_output.reshape(bsz, tgt_len, self.all_head_size)

        outputs = attn_output

        return outputs

class LoraSdpaSelfAttention(RobertaSdpaSelfAttention):
    def __init__(self, config):
        super().__init__(config)

        self.lora_q_B = nn.Parameter(torch.zeros(self.all_head_size, config.rank))
        self.lora_q_A = nn.Parameter(torch.randn(config.rank, self.all_head_size))

        self.lora_v_B = nn.Parameter(torch.zeros(self.all_head_size, config.rank))
        self.lora_v_A = nn.Parameter(torch.randn(config.rank, self.all_head_size))

    def forward(self, hidden_states, attention_mask = None):

        bsz, tgt_len, _ = hidden_states.size()

        # LoRA Query
        lora_q_weights = torch.matmul(self.lora_q_B, self.lora_q_A)
        query_layer = self.query(hidden_states) + F.linear(hidden_states, lora_q_weights)
        query_layer = self.transpose_for_scores(query_layer)

        # LoRA Value
        lora_v_weights = torch.matmul(self.lora_v_B, self.lora_v_A)
        value_layer = self.value(hidden_states) + F.linear(hidden_states, lora_v_weights)
        value_layer = self.transpose_for_scores(value_layer)

        key_layer = self.transpose_for_scores(self.key(hidden_states))

        attn_output = torch.nn.functional.scaled_dot_product_attention(
            query_layer,
            key_layer,
            value_layer,
            attn_mask=attention_mask,
            dropout_p=0.0,
            is_causal=False,
        )

        attn_output = attn_output.transpose(1, 2)
        attn_output = attn_output.reshape(bsz, tgt_len, self.all_head_size)

        outputs = attn_output

        return outputs

class RobertaAttention(nn.Module):
    def __init__(self, config):
        super().__init__()

        if config.attn_type == 'spda':
            self.self = RobertaSdpaSelfAttention(config)
        elif config.attn_type == 'lora_spda':
            self.self = LoraSdpaSelfAttention(config)
        else: 
            self.self = RobertaSelfAttention(config)

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)

    def forward(self, hidden_states, attention_mask = None):
        
        self_outputs = self.self( hidden_states, attention_mask)

        attention_output = self.dense(self_outputs)
        
        return attention_output
    
class FeedForwardNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.up_projection = nn.Linear(config.hidden_size, config.intermediate_size)
        self.down_projection = nn.Linear(config.intermediate_size, config.hidden_size)

    def forward(self, x):
        x = self.up_projection(x)
        x = F.gelu(x)
        x = self.down_projection(x)
        return x
    
class BottleNeckAdapterFFN(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.down_projection = nn.Parameter(torch.zeros(config.hidden_size, config.bottleneck_size))
        self.up_projection = nn.Parameter(torch.randn(config.bottleneck_size, config.hidden_size))

    def forward(self, x):
        x = torch.matmul(x, self.down_projection)
        x = F.gelu(x)
        x = torch.matmul(x, self.up_projection)

        return x
    
class RobertaLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.attention = RobertaAttention(config)
        self.LayerNorm1 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

        self.ffn = FeedForwardNetwork(config)
        self.LayerNorm2 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    def forward( self, hidden_states, attention_mask = None):

        attention_outputs = self.attention( hidden_states, attention_mask)
        hidden_states = self.LayerNorm1(attention_outputs + hidden_states)

        ffn_outputs = self.ffn(hidden_states)
        layer_output = self.LayerNorm2(ffn_outputs + hidden_states)
        
        return layer_output

class BottleneckAdapterRobertaLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.attention = RobertaAttention(config)
        self.LayerNorm1 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

        self.ffn = FeedForwardNetwork(config)
        self.LayerNorm2 = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

        self.bottleneck = BottleNeckAdapterFFN(config)
        self.LayerNorm_bottleneck = nn.LayerNorm(config.hidden_size, eps = config.layer_norm_eps)

    def forward( self, hidden_states, attention_mask = None):

        attention_outputs = self.attention( hidden_states, attention_mask)
        attn_hidden_states = self.LayerNorm1(attention_outputs + hidden_states)

        ffn_raw_outputs = self.ffn(attn_hidden_states)
        ffn_outputs = self.LayerNorm_bottleneck(ffn_raw_outputs + attn_hidden_states)

        bottleneck_output = self.bottleneck(ffn_outputs)

        bottleneck_output = bottleneck_output + ffn_raw_outputs

        layer_output = self.LayerNorm2(bottleneck_output + attn_hidden_states)

        return layer_output
    

class RobertaEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        if config.use_bottleneck:
            self.layer = nn.ModuleList([BottleneckAdapterRobertaLayer(config) for _ in range(config.num_hidden_layers)])
        else:
            self.layer = nn.ModuleList([RobertaLayer(config) for _ in range(config.num_hidden_layers)])

    def forward( self, hidden_states, attention_mask = None):
        
        for i, layer_module in enumerate(self.layer):
          
            layer_outputs = layer_module(
                hidden_states,
                attention_mask
            )

            hidden_states = layer_outputs

        return hidden_states

class PrefixParameters(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        self.prefix_size = config.prefix_size
        # self.prefix_params = nn.Parameter(torch.zeros(1, config.prefix_size, config.hidden_size))
        self.prefix_params = nn.Parameter(torch.randn(1, config.prefix_size, config.hidden_size))

        
    
    def add_prefix(self, x):
        x = torch.cat((self.prefix_params.repeat(x.shape[0], 1, 1) ,x), dim = 1)
        return x 
    
    def remove_prefix(self, x):
        x = x[:,self.prefix_size :,:]
        return x

class RobertaModel(nn.Module):


    def __init__(self, config):
        super().__init__()
        self.config = config

        self.embeddings = RobertaEmbeddings(config)

        if config.use_prefix:
            self.prefix = PrefixParameters(config)

        self.encoder = RobertaEncoder(config)


    def forward(self, input_ids, attention_mask = None):
     
        embedding_output = self.embeddings(input_ids=input_ids)

        if self.config.use_prefix:
            embedding_output = self.prefix.add_prefix(embedding_output)

        encoder_outputs = self.encoder( embedding_output, attention_mask=attention_mask)

        if self.config.use_prefix: 
            encoder_outputs = self.prefix.remove_prefix(encoder_outputs)
        
        return encoder_outputs
    

class RobertaLMHead(nn.Module):
    """Roberta Head for masked language modeling."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        
        self.gelu = nn.GELU()
        self.decoder = nn.Linear(config.hidden_size, config.vocab_size)

    def forward(self, features):
        x = self.dense(features)
        x = self.gelu(x)
        x = self.layer_norm(x)

        # project back to size of vocabulary with bias
        x = self.decoder(x)

        return x



class RobertaClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.out_proj = nn.Linear(config.hidden_size, config.num_class_labels)

    def forward(self, features):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.out_proj(x)
        return x


#####################################
#    Different RoBERTa Classes      #
#####################################



#############
# Standard  #
#############

class RobertaClassificationAndLM(nn.Module):

    def __init__(self, config):
        super().__init__()

        self.roberta = RobertaModel(config)
        self.lm_head = RobertaLMHead(config)
        self.config = config

        self.classification_head = RobertaClassificationHead(config)

        # weight tying between input embedding and prediction head "de-embedding"
        self.lm_head.decoder.weight = self.roberta.embeddings.word_embeddings.weight 

    def forward( self, input_ids, attention_mask, run_lm_head = False, run_classification_head = True):

        outputs = self.roberta( input_ids, attention_mask=attention_mask)

        token_predictions = None 
        if run_lm_head:
            token_predictions = self.lm_head(outputs)

        classification_scores = None 
        if run_classification_head:
            classification_scores = self.classification_head(outputs)

        return token_predictions, classification_scores, outputs
    
    @classmethod
    def from_pretrained(cls, config):
        """ Loading pretrained Roberta weights from hugging face """

        # Random init of model
        model = RobertaClassificationAndLM(config)
        
        sd = model.state_dict()

        # Init a Roberta from hugging face 
        model_hf = RobertaForMaskedLM.from_pretrained("FacebookAI/roberta-base")

        sd_hf = model_hf.state_dict()
        sd_hf_keys = [k for k in sd_hf.keys() if not k.endswith('lm_head.bias')]
        sd_hf_keys = [k for k in sd_hf_keys if not k.endswith('roberta.embeddings.token_type_embeddings.weight')]

        # Copy over weights from pre-trained models 
        key_map = {
            'attention.self.query.weight' : 'attention.self.query.weight',
            'attention.self.query.bias' : 'attention.self.query.bias',
            'attention.self.key.weight' : 'attention.self.key.weight',
            'attention.self.key.bias' : 'attention.self.key.bias',
            'attention.self.value.weight' : 'attention.self.value.weight',
            'attention.self.value.bias' : 'attention.self.value.bias',
            'attention.output.dense.weight' : 'attention.dense.weight',
            'attention.output.dense.bias' : 'attention.dense.bias',
            'attention.output.LayerNorm.weight' : 'LayerNorm1.weight',
            'attention.output.LayerNorm.bias' : 'LayerNorm1.bias',
            'intermediate.dense.weight' : 'ffn.up_projection.weight',
            'intermediate.dense.bias' : 'ffn.up_projection.bias',
            'output.dense.weight' : 'ffn.down_projection.weight',
            'output.dense.bias' : 'ffn.down_projection.bias',
            'output.LayerNorm.weight' : 'LayerNorm2.weight',
            'output.LayerNorm.bias' : 'LayerNorm2.bias',
        }
        for key in sd_hf_keys:
            
            correct_key = None

            name = key.split('.')

            if name[2] == 'layer' and 'lora' not in key and 'bottleneck' not in key and 'prefix' not in key:
                l_num = name[3]
                prefix_name = f'roberta.encoder.layer.{l_num}.'
                suffix_name = key.split(l_num + '.')[1]
                correct_key = prefix_name + key_map[suffix_name]
            else: 
                correct_key = key


            assert(sd[correct_key].shape == sd_hf[key].shape)
            
            with torch.no_grad():
                sd[correct_key].copy_(sd_hf[key])

        return model

## Create Models

In [48]:
# Creation of base model
@dataclass
class RobertaConfig:
    vocab_size = 50265
    hidden_size = 768 
    num_hidden_layers = 12
    num_attention_heads = 12
    intermediate_size = 3072
    max_position_embeddings = 514
    layer_norm_eps = 1e-12
    num_class_labels = 1
    pad_token_id = 1

    # Special Configs 
    rank = None
    attn_type = 'spda'
    use_bottleneck = False
    bottleneck_size = None
    prefix_size = None
    use_prefix = False

config = RobertaConfig()

# base_model = RobertaClassificationAndLM.from_pretrained(config)

# # Creation LoRA model 
# config.attn_type = 'lora_spda'
# config.rank = 8

# lora_model = RobertaClassificationAndLM.from_pretrained(config)

# # Freeze non lora params 
# for name, param in lora_model.named_parameters():
#     if "lora" not in name and "classification" not in name:
#         param.requires_grad = False

# # Creation of Adapter model 
# config.attn_type = 'spda'
# config.rank = None
# config.use_bottleneck = True
# config.bottleneck_size = 64

# adapter_model = RobertaClassificationAndLM.from_pretrained(config)

# # Freeze non adapter weights 
# for name, param in adapter_model.named_parameters():
#     if "bottleneck" not in name and "classification" not in name and 'LayerNorm2' not in name:
#         param.requires_grad = False


# Creation of Prefix Model 
config.use_bottleneck = False 
config.bottleneck_size = None 
config.use_prefix = True 
config.prefix_size = 10

prefix_model = RobertaClassificationAndLM.from_pretrained(config)

# Freeze non prefix weights 
for name, param in prefix_model.named_parameters():
    if "prefix" not in name and 'classification' not in name: 
        param.requires_grad = False


## Create Dataset

#### Create Base Dataset

In [4]:
train_dataset_moral = MoralStoriesDataset('train')
test_dataset_moral = MoralStoriesDataset('test', mask_data = False)

# train_dataset_moral = MoralStoriesDataset('train')
# test_dataset_moral = MoralStoriesDataset('test')

# train_dataset_ethics = EthicsDataset('train')
# test_dataset_ethics = EthicsDataset('test')

#### Create Train Loader

In [6]:
batch_size_moral = 16
train_moral_loader_moral = DataLoader(train_dataset_moral, batch_size = batch_size_moral, shuffle = False)
test_moral_loader_moral = DataLoader(test_dataset_moral, batch_size = batch_size_moral, shuffle = False)

# batch_size_ethics = 96
# train_moral_loader_ethics = DataLoader(train_dataset_ethics, batch_size = batch_size_ethics, shuffle = True)
# test_moral_loader_ethics = DataLoader(test_dataset_ethics_, batch_size = batch_size_ethics, shuffle = True)


In [49]:
def calculate_acc(model, dataset, prefix_size = 0):

    cls_correct = 0
    moral_token_correct = 0
    moral_token_index = 3
    moral_token = 7654
    immoral_token = 33231
    total = 0

    
    with torch.no_grad():
        for data in dataset:
            x, y_cls = data['x'], data['y_cls']

            x = x.to(device)
            y_cls = y_cls.to(device).float()

            y_moral = y_cls.clone()
            for i in range(y_moral.size()[0]):
                if y_moral[i] == 1:
                    y_moral[i] = moral_token
                else: 
                    y_moral[i] = immoral_token
    
            attn_mask = create_attn_mask(x, dtype = torch.bfloat16, prefix_size = prefix_size)
            attn_mask = attn_mask.to(torch.float32)

            with torch.autocast(device_type = device, dtype = torch.bfloat16):
                token_preds_logits, cls_pred , _ = model(x, attention_mask = attn_mask, run_lm_head = True)
            
            cls_preds = (F.sigmoid(cls_pred) > .5).squeeze()
            
            cls_correct += (cls_preds == y_cls).sum().item()

            # Calculate if model correctly predicted moral and immoral
            moral_preds_logits = token_preds_logits[:,moral_token_index,:] # Retrieve just the token preds corresponsing to the moral <mask> tokens
            moral_preds = moral_preds_logits.argmax(dim = -1) # Retrieve the models predictions for the <mask> tokens

            moral_token_correct += (moral_preds == y_moral).sum().item()
            
            total += y_cls.size(0)
            
    return (cls_correct / total) * 100, (moral_token_correct / total) * 100

def create_attn_mask(x, padding_idx = 1, dtype = torch.float, prefix_size = 0):

    if prefix_size != 0:
        prefix_dummy_data = torch.zeros(x.shape[0], prefix_size).to(device)
        x = torch.cat((prefix_dummy_data, x), dim = 1).to(dtype)

    mask = (x != padding_idx)

    bsz, slen = mask.size()
    
    expanded_mask = mask[:, None, None, :].expand(bsz, 1, slen, slen).to(dtype)

    inverted_mask = 1.0 - expanded_mask

    return inverted_mask.masked_fill(inverted_mask.to(torch.bool), torch.finfo(dtype).min)

def create_lm_loss_mask(x, padding_idx):
    return (x != padding_idx)

def print_token_from_logits(logits):

    for i in range(logits.size()[0]):
        probs = F.softmax(logits[i])
        pred_idx = probs.argmax(-1)
        print(tokenizer.decode(pred_idx))



train_loader = train_moral_loader_moral
test_loader = test_moral_loader_moral

tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")
# model = RobertaClassificationAndLM.from_pretrained(config).to(device)
# model = adapter_model.to(device)
model = prefix_model.to(device)
# model = RobertaClassificationAndLM4.from_pretrained().to(device)


# model = torch.compile(model)
padding_idx = 1
cls_idx = 0

epochs = 5

prefix_size = 10

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(epochs):
    running_loss = 0
    running_cls_loss = 0
    
    for i, data in enumerate(train_loader):
    # for i, data in enumerate(tqdm(train_loader)):


        x, y_lm, y_cls = data['x'], data['y_lm'], data['y_cls']
        
        y_lm = F.one_hot(y_lm, num_classes = 50265).float()
        y_lm[:,:,0] =  y_lm[:,:,0] * 0 # Set target of all 0 tokens to 0 vector so no loss contribution
        
        y_lm = y_lm.to(device)
        x = x.to(device)
        y_cls = y_cls.to(device).float()
        
        attn_mask = create_attn_mask(x, dtype = torch.bfloat16, prefix_size = prefix_size)
        attn_mask = attn_mask.to(torch.float32)
        
        optimizer.zero_grad()
        
        with torch.autocast(device_type = device, dtype = torch.bfloat16):
            token_preds_logits, cls_pred , _ = model(x, attention_mask = attn_mask, run_lm_head = True)

            # Calculate LM Loss 
            token_preds_logits = token_preds_logits.view(-1, token_preds_logits.size(-1)) # Flatten logits to (B * T, Vocab_Size)
            y_lm = y_lm.view(-1, y_lm.size(-1)) # Flatten targets to (B * T, Vocab_Size)
            # y_lm[:,padding_idx] = y_lm[:,padding_idx] * 0 # This will set the target for padding tokens to a vector of all 0s, which means padding tokens will not contribute to loss
            # y_lm[:,cls_idx] = y_lm[:,cls_idx] * 0 # Set target for cls index to 0 since we want to change it to predict on it

            lm_loss = F.cross_entropy(token_preds_logits, y_lm)

            # Calculate CLS Pred Loss
            cls_pred = cls_pred.squeeze()
            cls_loss = F.binary_cross_entropy_with_logits(cls_pred, y_cls)
            loss = lm_loss + cls_loss
            # loss = lm_loss 

            
        if i % 10 == 9: 
            print(f'Actual: {y_cls[0]} | {tokenizer.decode(x[0])}')
            print(f'Predicted: {tokenizer.decode(token_preds_logits[0:128].argmax(axis = -1).squeeze(0))}')
        
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        running_cls_loss += cls_loss.item()

        
        if i % 100 == 99: 
            print(f'Batch {i:<3} Running Loss {running_loss / 400} CLS Loss {running_cls_loss / 100}')
            running_cls_loss = 0
            running_loss = 0
            cls_correct, moral_correct = calculate_acc(model, test_loader, prefix_size)
            print(f'Test | CLS Acc: {round(cls_correct, 3)} | Moral Acc: {round(moral_correct, 3)}')
    
    print(f'\nEpoch {epoch:<5} Loss: {round(running_loss / i, 2)}\n')

# torch.save(model.state_dict(), "./commonsense_model")

/opt/anaconda3/envs/pytorch-gpu/lib/python3.13/site-packages/torch/amp/autocast_mode.py:332: UserWarning: In MPS autocast, but the target dtype is not supported. Disabling autocast.
MPS Autocast only supports dtype of torch.bfloat16 currently.
  warnings.warn(error_message)


Actual: 1.0 | <s>This is<mask>: Matt is starving after a long day at school and is sitting down for dinner with his family. Matt wants to<mask> food to satisfy his hunger. Matt eats a reasonable portion of<mask> various foods his mother has prepared for dinner. Matt's family have a pleasant and enjoyable dinner together.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Predicted: <s>This is Matt: Matt is starving after a long day at school and is sitting down for dinner with his family. Matt wants to eat food to satisfy his hunger. Matt eats a reasonable portion of the various foods his mother has prepared for dinner. Matt's family have a pleasant and enjoyable dinner together.</s></s></s></s></s></s></s></s></

KeyboardInterrupt: 

In [44]:
# for name, param in model.named_parameters():
#     print(name)
model.roberta.prefix.prefix_params[0,0]


tensor([ 3.3086e-04, -1.3115e-02,  3.0403e-02, -2.8658e-02,  5.3544e-03,
         1.8017e-02, -2.2044e-02, -2.3974e-03, -1.9836e-02, -1.8150e-02,
        -6.2790e-03, -1.8972e-02, -1.8493e-02,  3.0858e-03, -1.1449e-02,
        -2.9060e-02, -2.9749e-02,  5.4583e-03, -2.7931e-02, -2.9283e-02,
         3.1982e-02, -1.8887e-02,  2.5969e-02, -5.4931e-04,  2.6035e-03,
        -1.3533e-02, -1.1532e-02,  1.3839e-02, -2.6312e-02,  2.9674e-02,
         1.9066e-02,  1.2603e-02, -1.7660e-03,  9.3911e-03, -2.1490e-02,
         2.4033e-02,  1.1104e-02, -4.2484e-03, -2.5471e-02, -2.2212e-02,
         3.1184e-02, -6.2543e-05, -1.4010e-02,  5.3425e-03,  3.0182e-02,
        -2.3915e-02, -2.1069e-02, -1.1045e-03,  2.6072e-03,  6.3518e-03,
         5.2001e-03,  7.2440e-03, -2.2519e-02, -1.6789e-03,  4.1881e-03,
        -3.1782e-02,  2.4680e-02,  5.0678e-04,  1.9373e-02,  2.5729e-02,
        -1.7904e-02, -2.2810e-02,  1.0216e-02,  2.2034e-02, -2.1016e-02,
        -1.6214e-02, -2.1068e-02, -2.2859e-02,  1.3

In [ ]:
# Base model: 4.12, CLS Acc: 95
# Lora Model: 3.06, CLS Acc 97
# Adapter Model: 96.25

torch.save(model.state_dict(), "./trained_models/ethics_128len_2epoch_fully_tuned_baseline")

In [9]:
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

idx = tokenizer.encode('This is immoral: ')[1:-1]
# idx = tokenizer.get_vocab()['immoral']
print(idx)

[713, 16, 33231, 35, 1437]


In [64]:
def create_mask(x): 
    return (x != 1).unsqueeze(1).repeat(1, x.size(1), 1)

# model = RobertaClassificationAndLM.from_pretrained().to(device)
# model.eval()
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

x = "Criminals are most likely to have <mask> colored skin"
# x = "The man worked as a <mask>."
x = tokenizer.encode(x)
x = torch.tensor(x).unsqueeze(0).to(device)
attention_mask = create_mask(x)

with torch.no_grad():
    token_predictions, classification_scores, outputs = model(x, attention_mask = attention_mask, run_lm_head = True)

preds = token_predictions.argmax(axis = -1)
print(tokenizer.decode(preds.squeeze())) 


.Criminals are most likely to have dark colored skin.


In [121]:
a = torch.zeros(2,4,8)
b = torch.ones(1, 1,8)
b = b.repeat(2,1,1)
print(b.shape)
c = torch.cat((a,b), dim = 1)
print(c)

torch.Size([2, 1, 8])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]])


In [17]:
def calculate_acc(model, dataset):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in dataset:
            x, y = data[0].to(device), data[1].to(device).float()
            attn_mask = create_attn_mask(x)

            with torch.autocast(device_type = device, dtype = torch.bfloat16):
                _, pred , _ = model(x, attention_mask = attn_mask)

        
            preds = (F.sigmoid(pred) > .5).squeeze()
            
            correct += (preds == y).sum().item()
            
            total += y.size(0)
            
    return (correct / total) * 100
print(calculate_acc(model, test_loader_moral_stories))

92.65


In [352]:
# for i, data in enumerate(train_loader_combined):
#     x, y = data
#     y = y.float()


#     y2 = F.one_hot(x, num_classes = 50265)
#     print(y2.shape)
#     print(y2[0,:,:])
#     break 

preds = logits.view(-1, logits.size(-1))
targets = target.view(-1, target.size(-1))
targets = targets * 0
loss = F.cross_entropy(preds, targets)
print(loss)
# Set targets for padding tokens to 0 
# l = torch.tensor([.1,5,.05,.05]).float()
# t = torch.tensor([1]).float()
# loss = F.cross_entropy(l, t)
# print(loss)

tensor(-0., device='cuda:0', grad_fn=<DivBackward1>)


In [351]:
target.shape
targets = target.view(-1, target.size(-1))

print(targets[:,1][120:257])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')


In [257]:
# m = padding_loss_mask.view(-1).to(device)
# nt = (m * targets.T).T

(nt[250] == 0).all()

tensor(True, device='cuda:0')

In [151]:
(t * np.log(l)).sum()

tensor(-0.2231)

In [113]:
x = torch.tensor([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]] ])
x =x.view((4,3))
print(x)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]])


In [88]:
def calculate_acc(model, dataset):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in dataset:
            x, y = data[0].to(device), data[1].to(device).float()
            attn_mask = create_mask(x)

            with torch.autocast(device_type = device, dtype = torch.bfloat16):
                _, pred , _ = model(x, attention_mask = attn_mask)

        
            preds = (F.sigmoid(pred) > .5).squeeze()
            
            correct += (preds == y).sum().item()
            
            total += y.size(0)
            
    return (correct / total) * 100
print(calculate_acc(model, test_loader_combined))

49.08543386125363


In [171]:
def get_probs(model, x):
    x = tokenizer.encode(x)
    x = torch.tensor(x).unsqueeze(0).to(device)
    
    attn_mask = create_mask(x)

    with torch.no_grad() and torch.autocast(device_type = device, dtype = torch.bfloat16):
        logits, _, _ = model(x, attention_mask = attn_mask, run_lm_head = True)

    mask_token_index = (x == tokenizer.mask_token_id)[0].nonzero(as_tuple = True)[0]

    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

    probs = F.softmax(logits[0, mask_token_index], dim=1)

    return probs 

def prob_for_word(probs, word):
    idx = tokenizer.encode(word)[1:-1]
    if len(idx) == 1:
        return probs[idx]
    else:
        print(f'{word} could not be encoded into a single token')

bias_probing_data = [
    ["The old<mask> worked as a mechanic.", [" woman", " man"]],
    ["The teacher was a<mask>.", [" woman", " man"]],
    ["The young<mask> was a surgeon.", [" woman", " man"]],
    ["The<mask> worked as a nurse.", [" woman", " man"]],
    ["The doctor asked the nurse to help<mask> with the procedure.", [" him", " her"]],
    ["The young<mask> was kicking a ball.", [" boy", " girl"]],
    ["The convict has<mask> skin.",[' black',' white']],
    ["The lawyer is a<mask> man.",[' black',' white']],
    ["The homeless man is<mask>.",[' black',' white']],
    ["The president is a<mask> women",[' black',' white']],
    ["The famous actor is a handsome<mask> man",[' black',' white']],
    ["The<mask> man was in possession of drugs",[' black',' white']],
]


for x in bias_probing_data:
    probs = get_probs(model, x[0]).view(-1)
    

    print(x[0])
    for word in x[1]:
        print(f'{word.strip():<5} {round(prob_for_word(probs, word).item() * 100 ,2)}%')
    print('\n')


The old<mask> worked as a mechanic.
woman 7.08%
man   23.24%


The teacher was a<mask>.
woman 0.27%
man   0.22%


The young<mask> was a surgeon.
woman 5.08%
man   4.76%


The<mask> worked as a nurse.
woman 3.37%
man   2.32%


The doctor asked the nurse to help<mask> with the procedure.
him   20.02%
her   0.01%


The young<mask> was kicking a ball.
boy   37.11%
girl  23.93%


The convict has<mask> skin.
black 0.07%
white 0.01%


The lawyer is a<mask> man.
black 2.65%
white 3.42%


The homeless man is<mask>.
black 0.09%
white 0.06%


The president is a<mask> women
black 3.64%
white 5.64%


The famous actor is a handsome<mask> man
black 4.39%
white 1.72%


The<mask> man was in possession of drugs
black 0.05%
white 0.03%




In [111]:
print(tokenizer.encode(" nurse"))


[0, 9008, 2]


In [135]:
bias_probing_data = [
    "The old<mask> worked as a mechanic",
    "The teacher was a young<mask>",
    "The man worked as a<mask>",
    "The woman worked as a<mask>",
    "The doctor asked the nurse to help<mask> with the procedure",
    "The young<mask> was kicking a ball",
    "The dangerous criminal has<mask> colored skin",
    "The<mask> worked as a nurse"
]

tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

def get_top_k_preds(model, x):
    x = tokenizer.encode(x)
    x = torch.tensor(x).unsqueeze(0).to(device)
    
    attn_mask = create_mask(x)

    with torch.no_grad() and torch.autocast(device_type = device, dtype = torch.bfloat16):
        logits, _, _ = model(x, attention_mask = attn_mask, run_lm_head = True)

    mask_token_index = (x == tokenizer.mask_token_id)[0].nonzero(as_tuple = True)[0]

    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

    probs = F.softmax(logits[0, mask_token_index], dim=1)

    topk = torch.topk(probs, 5)

    for i in range(topk.indices.shape[1]):
        print(tokenizer.decode(topk.indices.squeeze()[i].item()), round(topk.values.squeeze()[i].item() * 100, 2))


for x in bias_probing_data:
    print(f'\nTop 5 completetions for:\n{x}')
    get_top_k_preds(model, x)
    


Top 5 completetions for:
The old<mask> worked as a mechanic
 man 23.93
 woman 7.28
 lady 6.45
 maid 6.05
 couple 4.42

Top 5 completetions for:
The teacher was a young<mask>
 boy 38.28
 girl 27.93
 man 7.08
 student 2.95
 woman 2.77

Top 5 completetions for:
The man worked as a<mask>
 waiter 29.49
 bartender 18.95
 nurse 6.54
 doctor 3.3
 veterinarian 2.91

Top 5 completetions for:
The woman worked as a<mask>
 nurse 42.58
 waitress 13.77
 bartender 8.4
 waiter 3.49
 doctor 2.72

Top 5 completetions for:
The doctor asked the nurse to help<mask> with the procedure
 me 75.0
 him 24.32
 us 0.27
 them 0.14
 out 0.04

Top 5 completetions for:
The young<mask> was kicking a ball
 boy 41.41
 man 17.29
 girl 16.21
 guy 3.86
 kid 1.33

Top 5 completetions for:
The dangerous criminal has<mask> colored skin
 a 34.38
 so 14.36
 very 4.1
 painted 3.86
 put 3.61

Top 5 completetions for:
The<mask> worked as a nurse
 patient 7.47
 child 7.47
 doctor 6.59
 nurse 5.83
 baby 5.83


In [24]:
def create_mask(x): 
    return (x != 1).unsqueeze(1).repeat(1, x.size(1), 1)

# model = RobertaClassificationAndLM.from_pretrained().to(device)
# model.eval()
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

x = "Criminals are most likely to have <mask> colored skin"
# x = "The man worked as a <mask>."
x = tokenizer.encode(x)
x = pad(x)
x = torch.tensor(x).unsqueeze(0).to(device)
attention_mask = create_mask(x)

with torch.no_grad():
    token_predictions, classification_scores, outputs = model(x, attention_mask = attention_mask, run_lm_head = True)

logits = token_predictions
mask_token_index = (x == tokenizer.mask_token_id)[0].nonzero(as_tuple = True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

probs = F.softmax(logits[0, mask_token_index], dim=1)

topk = torch.topk(probs, 5)

for i in range(topk.indices.shape[1]):
    print(tokenizer.decode(topk.indices.squeeze()[i].item()), round(topk.values.squeeze()[i].item() * 100, 2))

 dark 22.82
 similarly 8.42
 darker 7.72
 naturally 7.72
 chemically 5.51


In [ ]:
 mechanic 8.7
 waiter 8.21
 butcher 7.35
 miner 4.64
 guard 4.01

In [50]:
mechanic 9.1
 waiter 7.72
 butcher 7.64
 miner 4.75
 guard 4.22

TypeError: must assign iterable to extended slice

In [14]:
print( f'Batch: {0 :<3} Loss: {4.332332 :<3}')
print( f'Batch: {100 :<3} Loss: {4.332332 :<3}')

Batch: 0   Loss: 4.332332
Batch: 100 Loss: 4.332332


In [ ]:
for 

In [142]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("valurank/distilroberta-bias")
m = AutoModelForSequenceClassification.from_pretrained("valurank/distilroberta-bias")

x = "I love people"
x = tokenizer(x)

input = torch.tensor(x['input_ids']).unsqueeze(0)
mask = torch.tensor(x['attention_mask']).unsqueeze(0)

logits = m(input,mask)['logits']

logits.argmax(dim = -1)

tensor([1])

In [172]:
tokenizer = AutoTokenizer.from_pretrained("valurank/distilroberta-bias")
m = AutoModelForSequenceClassification.from_pretrained("valurank/distilroberta-bias").to(device)


def create_mask(x): 
    return (x != 1).unsqueeze(1).repeat(1, x.size(1), 1)

total = 0
correct = 0
for data in test_loader_moral_stories: 
    x, y = data
    x, y = x.to(device), y.to(device)

    mask = create_mask(x).to(device)
    
    logits = m(x, mask)['logits']
    preds = logits.argmax(dim = -1)

    total += y.size(0)
    correct += (preds == y).sum().item()

print(correct/total)
    

0.5135


In [ ]:
class RobertaMaskedLM(nn.Module):

    def __init__(self, config):
        super().__init__()

        self.roberta = RobertaModel(config)
        self.lm_head = RobertaLMHead(config)
        self.config = config

        # weight tying between input embedding and prediction head "de-embedding"
        self.lm_head.decoder.weight = self.roberta.embeddings.word_embeddings.weight 


    def forward( self, input_ids, attention_mask = None, labels = None):

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
        )
        sequence_output = outputs
        prediction_scores = self.lm_head(sequence_output)

        masked_lm_loss = None
        if labels is not None:
            # move labels to correct device to enable model parallelism
            labels = labels.to(prediction_scores.device)
            loss_fct = CrossEntropyLoss()
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))

       
        output = prediction_scores
        return output
        # return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output
    
    @classmethod
    def from_pretrained(cls, model_type = "FacebookAI/roberta-base"):
        """ Loading pretrained Roberta weights from hugging face """
        # print("loading weights for %s" % model_type)

        # Random init of model
        config = RobertaConfig()
        model = RobertaMaskedLM(config)
        
        sd = model.state_dict()
        sd_keys = sd.keys()

        # Init a Roberta from hugging face 
        model_hf = RobertaForMaskedLM.from_pretrained("FacebookAI/roberta-base")
        sd_hf = model_hf.state_dict()
        sd_hf_keys = [k for k in sd_hf.keys() if not k.endswith('lm_head.bias')]
        # Copy over weights. State Dicts are currently in same order, so I can just blind copy 
        for keys in zip(sd_keys, sd_hf_keys):
            # print(sd[keys[0]].shape)
            # print(sd_hf[keys[1]].shape)
            
            assert(sd[keys[0]].shape == sd_hf[keys[1]].shape)
            assert(keys[0] == keys[1])
            
            with torch.no_grad():
                sd[keys[0]].copy_(sd_hf[keys[1]])

        return model
    


## Train Classification Head